In [1]:
import re
import json
import requests
import os
import time

In [2]:
# ../datas/item_pages/の一覧を取得する
directory_path = '../datas/html_recipes/item_pages/Vegetable_Recipes'
output_path = '../datas/html_recipes/out_jsons/'
output_file_name = f'{output_path}Vegetable_Recipes.json'
prompt_template = "What do you cock with beef?"

files = os.listdir(directory_path)

In [3]:
def replace_unicode_in_json(data):
    if isinstance(data, dict):
        return {key: replace_unicode_in_json(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [replace_unicode_in_json(item) for item in data]
    elif isinstance(data, str):
        return data.encode('utf-8').decode('unicode_escape').replace("®", "")
    else:
        return data

In [4]:
def parse_recipe_json(file_path):
    file_path = f'{directory_path}/{file_path}'  # HTMLファイルのパスを指定

    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    # <script>タグ内のJSONデータを正規表現で抽出
    pattern = r'<script id="allrecipes-schema_1-0"[^>]*type="application/ld\+json"[^>]*>(.*?)</script>'
    match = re.search(pattern, html_content, re.DOTALL)
    json_content = match.group(1).strip()
    json_data = json.loads(json_content)

    item_result = {}
    for index in range(len(json_data)):
        item = json_data[index]
        
        instructions = []

        for index2 in range(len(item.get("recipeInstructions", []))):
            instructions.append(item.get("recipeInstructions")[index2].get("text"))
        
        item_result = {
            "id": "",
            "title": replace_unicode_in_json(item.get("name", "")),
            "prompt": prompt_template,
            "ingredients": replace_unicode_in_json(item.get("recipeIngredient")),
            "instructions": replace_unicode_in_json(instructions)
            
        }
    return item_result

In [5]:
json_result = []
for file_path in files:
    item_result = parse_recipe_json(file_path)
    json_result.append(item_result)
    
print(len(json_result))

68


In [6]:
with open(output_file_name, "w") as f:
    output = json.dumps(json_result, ensure_ascii=False, indent=4)
    f.write(output)